In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from IPython.display import Image
from urllib.request import urlopen
import pandas as pd
import re
import time

In [22]:
# 웹드라이브로 크롬브라우즈 띄운다.
class OpenDriverMovieSite():
    driver_path = "../driver/chromedriver.exe"
    driver = webdriver.Chrome(executable_path=driver_path)
    url_page = 'https://movie.naver.com/movie/point/af/list.nhn'
    movieName = ''
    allReviews = []
    allReviewsIndex = []
    a = 1

    def __init__(self):
        # 클래스를 실행하면 홈페이지로 이동한다.
        self.driver.get(self.url_page)
        time.sleep(1)

    def ReadCSV(self, movie2018):
        movie_name = movie2018['영화명']
        movie_name[1]
        for i in range(0, len(movie2018)):  # CSV 파일의 Rows 갯수를 뒤에 넣어주세여 (1, Rows갯수)
            #             print(self.movieName)
            print(movie_name[i])
            self.movieName = movie_name[i]
            self.BasicOpen()
            self.InputMovieTitle(movie_name[i])

    def BasicOpen(self):
        # 켜진 홈페이지에서 관련영화를 클릭한 후 새로운 창으로 전환하는 것까지
        # 영화명이 지속적으로 바뀔 것이므로 BasicOpen 함수를 활용
        self.driver.find_element_by_xpath('//*[@id="old_content"]/fieldset/form/select').click()
        time.sleep(0.5)
        self.driver.find_element_by_xpath('//*[@id="old_content"]/fieldset/form/select/option[2]').click()
        time.sleep(0.5)
        self.driver.find_element_by_xpath('//*[@id="old_content"]/fieldset/form/input[3]').click()
        time.sleep(0.5)
        self.driver.switch_to.window(self.driver.window_handles[-1])
        self.driver.find_element_by_xpath(
            '//*[@id="page_content"]/div/div/div/div/form/table/tbody/tr/td/input[1]').click()
        time.sleep(0.5)

    def InputMovieTitle(self, movie_title):
        self.driver.find_element_by_css_selector(
            '#page_content > div > div > div > div > form > table > tbody > tr > td > input.input_type_text_1').send_keys(
            movie_title)
        time.sleep(0.5)
        self.driver.find_element_by_xpath(
            '//*[@id="page_content"]/div/div/div/div/form/table/tbody/tr/td/input[2]').click()
        time.sleep(0.5)
        self.driver.find_element_by_xpath('//*[@id="page_content"]/div/div/div[2]/table/tbody/tr[2]/td/a/img').click()
        time.sleep(0.5)
        self.driver.switch_to.window(self.driver.window_handles[0])
        time.sleep(0.5)
        self.driver.find_element_by_xpath('//*[@id="old_content"]/fieldset/form/input[4]').click()

        html = urlopen(self.driver.current_url)
        soup = BeautifulSoup(html, "lxml")
        # 코드값 찾아주기
        pattern = re.compile("\d+")
        result = pattern.findall(self.driver.current_url)
        print(result)
        code = result[0] if len(result) > 0 else ''

        # 리뷰의 총개수
        num = soup.find("strong", "c_88 fs_11").get_text()
        num_total = int(num.replace(',', ''))
        last_page = num_total // 10 + 1
        temp = "번호 : {} // 영화명 : {} // code : {} // num_total : {} // last_page : {}".format(self.a, self.movieName,
                                                                                             code, num_total, last_page)
        self.allReviewsIndex.append(temp)
        #         print("번호 : {} // 영화명 : {} // code : {} // num_total : {} // last_page : {}".format(self.a, self.movieName, code, num_total, last_page))
        self.a = self.a + 1

        #         last_page = 1000 if last_page > 1000 else last_page
        last_page = 1000 if last_page > 1000 else last_page

        self.GetReviews(code, num_total, last_page)

    def GetReviews(self, code, num_total, last_page):
        tmp = []
        for i in range(last_page):

            # 각 리뷰 페이지로 이동해서 그 url로 드라이버 창 넘어가고-> 수프화
            url_page = 'https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword={}&target=&page={}'.format(code,
                                                                                                                  i + 1)
            print("\"{}\"의 {} 페이지를 수집하고 있습니다..".format(self.movieName, url_page))
            html = urlopen(url_page)
            soup = BeautifulSoup(html, "lxml")
            # 리뷰들 뽑아줌
            review = soup.find_all("td", "title")
            for i in range(len(review)):
                review[i] = review[i].get_text()
                review[i] = review[i].replace('\n', '')
                review[i] = review[i].replace(self.movieName, '')  # 왜 영화명을 '' 처리하는지 잘 모르겠지만 우선 둠
                review[i] = review[i].replace('\r', '')
                review[i] = review[i].replace('\t', '')
                review[i] = review[i].replace('신고', '')
                tmp.append(review[i])
        self.allReviews.append(tmp)


if __name__ == "__main__":
    opendrivermoviesite = OpenDriverMovieSite()
    movie2018 = pd.read_csv('data/awarded.csv', sep=',', encoding='euc-kr')
    opendrivermoviesite.ReadCSV(movie2018)

택시운전사
['146469']
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=1 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=2 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=3 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=4 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=5 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=6 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=7 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=8 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=9 페

"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=75 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=76 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=77 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=78 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=79 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=80 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=81 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=82 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=83 페이지를 수집하고

"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=148 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=149 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=150 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=151 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=152 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=153 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=154 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=155 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=156 

"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=221 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=222 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=223 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=224 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=225 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=226 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=227 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=228 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=229 

"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=294 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=295 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=296 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=297 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=298 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=299 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=300 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=301 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=302 

...

"감시자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=98146&target=&page=656 페이지를 수집하고 있습니다..
"감시자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=98146&target=&page=657 페이지를 수집하고 있습니다..
"감시자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=98146&target=&page=658 페이지를 수집하고 있습니다..
"감시자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=98146&target=&page=659 페이지를 수집하고 있습니다..
"감시자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=98146&target=&page=660 페이지를 수집하고 있습니다..
"감시자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=98146&target=&page=661 페이지를 수집하고 있습니다..
"감시자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=98146&target=&page=662 페이지를 수집하고 있습니다..
"감시자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=98146&target=&page=663 페이지를 수집하고 있습니다..
"감시자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=98146&target=&page=664 페이지를 수집하고 있습니다..
"

"감시자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=98146&target=&page=730 페이지를 수집하고 있습니다..
"감시자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=98146&target=&page=731 페이지를 수집하고 있습니다..
"감시자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=98146&target=&page=732 페이지를 수집하고 있습니다..
"감시자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=98146&target=&page=733 페이지를 수집하고 있습니다..
"감시자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=98146&target=&page=734 페이지를 수집하고 있습니다..
"감시자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=98146&target=&page=735 페이지를 수집하고 있습니다..
"감시자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=98146&target=&page=736 페이지를 수집하고 있습니다..
"감시자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=98146&target=&page=737 페이지를 수집하고 있습니다..
"감시자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=98146&target=&page=738 페이지를 수집하고 있습니다..
"

"감시자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=98146&target=&page=804 페이지를 수집하고 있습니다..
"감시자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=98146&target=&page=805 페이지를 수집하고 있습니다..
"감시자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=98146&target=&page=806 페이지를 수집하고 있습니다..
"감시자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=98146&target=&page=807 페이지를 수집하고 있습니다..
"감시자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=98146&target=&page=808 페이지를 수집하고 있습니다..
"감시자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=98146&target=&page=809 페이지를 수집하고 있습니다..
"감시자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=98146&target=&page=810 페이지를 수집하고 있습니다..
"감시자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=98146&target=&page=811 페이지를 수집하고 있습니다..
"감시자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=98146&target=&page=812 페이지를 수집하고 있습니다..
"

"감시자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=98146&target=&page=878 페이지를 수집하고 있습니다..
"감시자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=98146&target=&page=879 페이지를 수집하고 있습니다..
"감시자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=98146&target=&page=880 페이지를 수집하고 있습니다..
"감시자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=98146&target=&page=881 페이지를 수집하고 있습니다..
"감시자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=98146&target=&page=882 페이지를 수집하고 있습니다..
"감시자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=98146&target=&page=883 페이지를 수집하고 있습니다..
"감시자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=98146&target=&page=884 페이지를 수집하고 있습니다..
"감시자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=98146&target=&page=885 페이지를 수집하고 있습니다..
"감시자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=98146&target=&page=886 페이지를 수집하고 있습니다..
"

"감시자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=98146&target=&page=952 페이지를 수집하고 있습니다..
"감시자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=98146&target=&page=953 페이지를 수집하고 있습니다..
"감시자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=98146&target=&page=954 페이지를 수집하고 있습니다..
"감시자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=98146&target=&page=955 페이지를 수집하고 있습니다..
"감시자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=98146&target=&page=956 페이지를 수집하고 있습니다..
"감시자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=98146&target=&page=957 페이지를 수집하고 있습니다..
"감시자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=98146&target=&page=958 페이지를 수집하고 있습니다..
"감시자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=98146&target=&page=959 페이지를 수집하고 있습니다..
"감시자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=98146&target=&page=960 페이지를 수집하고 있습니다..
"

In [3]:
len(opendrivermoviesite.allReviews)

69

In [4]:
len(opendrivermoviesite.allReviewsIndex)

69

In [6]:
opendrivermoviesite.allReviews[68]

['여주인공 박지유만 보인다. 근데  갑툭튀인건지 정보가 없다 ', '킬링타임용으로 갠춘함 ']

In [8]:
opendrivermoviesite.allReviewsIndex

['번호 : 1 // 영화명 : 신과함께-인과 연 // code : 167697 // num_total : 39057 // last_page : 3906',
 '번호 : 2 // 영화명 : 신과함께-죄와 벌 // code : 85579 // num_total : 57393 // last_page : 5740',
 '번호 : 3 // 영화명 : 1987 // code : 158191 // num_total : 30997 // last_page : 3100',
 '번호 : 4 // 영화명 : 독전 // code : 158178 // num_total : 18140 // last_page : 1815',
 '번호 : 5 // 영화명 : 공작 // code : 153687 // num_total : 21689 // last_page : 2169',
 '번호 : 6 // 영화명 : 안시성 // code : 163533 // num_total : 31613 // last_page : 3162',
 '번호 : 7 // 영화명 : 그것만이 내 세상 // code : 158180 // num_total : 13991 // last_page : 1400',
 '번호 : 8 // 영화명 : 마녀 // code : 175322 // num_total : 23276 // last_page : 2328',
 '번호 : 9 // 영화명 : 탐정: 리턴즈 // code : 159892 // num_total : 9694 // last_page : 970',
 '번호 : 10 // 영화명 : 너의 결혼식 // code : 140652 // num_total : 15070 // last_page : 1508',
 '번호 : 11 // 영화명 : 곤지암 // code : 172454 // num_total : 15995 // last_page : 1600',
 '번호 : 12 // 영화명 : 지금 만나러 갑니다 // code : 168298 // num_total : 12910 // last_

In [9]:
# 데이터 폭발로 무척 느립니다. 가이드라인을 드릴테니 참고해서 활용하세요!

In [ ]:
# 1. allReviews의 리스트 내용을 잘 살펴보고 데이터 프레임으로 변환하는 것을 "다른 곳에서 작업해서 여기에 복붙해서 사용하세요"
# 2. 역시 allReviewsIndex 역시 위와 같이 사용하세요
# 3. 데이터 프레임으로 전환하였다면, csv파일로 변환시켜서 파일로 관리하세요
# 4. 이 csv 파일은 중요한 원본 데이터이니 꼭 백업을 해두세요
# 5. csv 파일을 생성했다면 이를 사용해서 데이터 분석을 진행하세요!

In [13]:
df_all = pd.DataFrame(opendrivermoviesite.allReviews)
df_all.to_csv('data/allreviews.csv', sep=',', encoding='euc-kr')

In [15]:
df_info =pd.DataFrame(opendrivermoviesite.allReviewsIndex)
df_info.to_csv('data/allReviewsIndex.csv', sep=',', encoding='euc-kr')

In [17]:
title = []
code = []
num_total= []
last_page = []
for i in range(len(opendrivermoviesite.allReviewsIndex)):
    y = opendrivermoviesite.allReviewsIndex[i].split("//")
    t = y[1].split(":")
    c = y[2].split(":")
    n = y[3].split(":")
    l = y[4].split(":")
    title.append(t[1])
    code.append(c[1])
    num_total.append(n[1])
    last_page.append(l[1])
title[:3] , code[:3],  num_total[:3], last_page[:3]

([' 신과함께-인과 연 ', ' 신과함께-죄와 벌 ', ' 1987 '],
 [' 167697 ', ' 85579 ', ' 158191 '],
 [' 39057 ', ' 57393 ', ' 30997 '],
 [' 3906', ' 5740', ' 3100'])

In [18]:
allreview_index = pd.DataFrame(({'제목':title, 
                          '코드':code, 
                          '총리뷰수':num_total, 
                          '마지막페이지':last_page}))

In [19]:
allreview_index

,제목,코드,총리뷰수,마지막페이지
0,신과함께-인과 연,167697,39057,3906
1,신과함께-죄와 벌,85579,57393,5740
2,1987,158191,30997,3100
3,독전,158178,18140,1815
4,공작,153687,21689,2169
5,안시성,163533,31613,3162
6,그것만이 내 세상,158180,13991,1400
7,마녀,175322,23276,2328
8,탐정,159892,9694,970
9,너의 결혼식,140652,15070,1508


In [20]:
allreview_index.to_csv('data/allreview_index_df.csv', sep=',', encoding='euc-kr')